# Hashing: Applications

You already know several applications of Python's set and dictionary (thus, hashing) in your programming life. 
For example, computing frequencies of distinct strings in text or numbers in a list.

Today we will see a very important application in managing data bases: 
- Data aggregation by grouping rows by values in some colum (aka **GroupBy**)
- Merging different datasets (aka **Join**) 

---

## Data Aggregation in Pandas: GroupBy

Groupby is a pretty easy concept: *we create a grouping of categories and apply an aggregation function to element of each category*.

It's a simple concept but it's an extremely valuable technique that's widely used in data science. The value of groupby really comes from its ability to **aggregate data efficiently**, both in performance and the amount code it takes. 

Let's first create a toy DataFrame

In [1]:
import random
import pandas as pd

random.seed(42)
    
# Random pets column
pet_list = ["cat", "dog", "alligator", "snake"]
pet = [random.choice(pet_list) for i in range(1,15)]

# Random weight of animal column
weight = [random.choice(range(5,15)) for i in range(1,15)]

# Random length of animals column
length = [random.choice(range(1,10)) for i in range(1,15)]

# random age of the animals column
age = [random.choice(range(1,5)) for i in range(1,15)]

# Put everyhting into a dataframe
df = pd.DataFrame()
df["animal"] = pet
df["age"] = age
df["weight"] = weight
df["length"] = length

df

,animal,age,weight,length
0,cat,1,13,6
1,cat,4,14,5
2,alligator,1,5,3
3,dog,3,13,4
4,dog,3,8,6
5,dog,2,13,2
6,cat,1,11,2
7,cat,1,8,7
8,snake,2,12,2
9,cat,3,14,6


### Groupby a variable

Groupby essentially splits the data into different groups *depending on a variable* (or more variables) of your choice. 

For example, the expression `df.groupby('animal')` splits the DataFrame by 'animal' category.

The `groupby()` function returns a *GroupBy* object, but essentially describes how the rows of the original dataset has been split. 

In the GroupBy object `groups` variable is a dictionary whose keys are the unique groups and corresponding values being the axis labels belonging to each group.

In [2]:
animal_groups = df.groupby('animal') # Group by animal category

print( type(animal_groups), end='\n\n')

print( animal_groups.groups, end='\n\n')

print( animal_groups.groups.keys())

<class 'pandas.core.groupby.generic.DataFrameGroupBy'>

{'alligator': [2], 'cat': [0, 1, 6, 7, 9, 10, 11], 'dog': [3, 4, 5, 12, 13], 'snake': [8]}

dict_keys(['alligator', 'cat', 'dog', 'snake'])


Once we have a grouping, we can apply an aggregation function to the groupby object to get the 
function applied to the elements of each group. 

For example,

```python
animal_groups = df.groupby("animal")

gdf = animal_groups.max()

```

computes the maximum value in each column of each group. 

There are several possible aggregation functions. For example, 

```text
mean()
std()
median()	
min()
max()
sum()
count()
```

In [3]:
animal_groups = df.groupby('animal')

gdf = animal_groups.max()

print( type(gdf) ) # gdf is a DataFrame

gdf 

<class 'pandas.core.frame.DataFrame'>


,age,weight,length
animal,,,
alligator,1,5,3
cat,4,14,7
dog,4,13,8
snake,2,12,2


#### Example
What's the average weight of each kind of animal?

To find the average weight of each category of animal, we group the animals by 'animal' and, then, we apply the *mean* function. 

In [4]:
print("Avg weight:", animal_groups.mean()['weight'])

animal_groups.mean()

Avg weight: animal
alligator     5.000000
cat          10.571429
dog          10.400000
snake        12.000000
Name: weight, dtype: float64


,age,weight,length
animal,,,
alligator,1.000000,5.000000,3.000000
cat,1.857143,10.571429,5.285714
dog,2.600000,10.400000,4.200000
snake,2.000000,12.000000,2.000000


#### Example
What's the overall length of snakes?

In [5]:
s = animal_groups.sum().loc['snake', 'length'] # Access snake row, length col

# Alternative: s = animal_groups['length'].sum().loc['snake'] # may be faster

print("Sum snakes' length:", s)


Sum snakes' length: 2


#### Example
What's the age of the oldest cat?

In [6]:
s = animal_groups['age'].max().loc['cat']

print("Age older cat:", s)

Age older cat: 4


### GroupBy more variables

It is also possible to group by more than one variable, allowing more complex queries.

For example,

```python
animalage_groups = df.groupby( ['animal', 'age'] )
```

groups first by 'animal' and then, by 'age' within each group.

Note the list of labels in calling `groupby`.

In [7]:
animalage_groups = df.groupby( ['animal', 'age'] )

animalage_groups.groups

{('alligator', 1): [2], ('cat', 1): [0, 6, 7, 10], ('cat', 2): [11], ('cat', 3): [9], ('cat', 4): [1], ('dog', 1): [12], ('dog', 2): [5], ('dog', 3): [3, 4], ('dog', 4): [13], ('snake', 2): [8]}

#### Example
What's the maximum weight of a 3-years old dog?

In [8]:
print( animalage_groups['weight'].max() )               # operation max () 
print()

print( animalage_groups['weight'].max().loc['dog'] )  # .loc['dog']
print()

print("3yo dog weight:", animalage_groups['weight'].max().loc[('dog', 3)] ) # Multiindex we use a pair 

animal     age
alligator  1       5
cat        1      13
           2       5
           3      14
           4      14
dog        1       7
           2      13
           3      13
           4      11
snake      2      12
Name: weight, dtype: int64

age
1     7
2    13
3    13
4    11
Name: weight, dtype: int64

3yo dog weight: 13


--- 
### Exercise: Implements your own GroupBy

In the rest of your life you are going to use GroupBy implemented in some library, but in this exercise we will implement our own simplified version. 

You are give a list of tuples, all with the same number of components. In our simplified implementation  of a pandas' DataFrame each tuple in the list is a row of the DataFrame. Each component of a tuple is a value of a column.

#### Part I
Our first goal is to implement an index to efficiently group by one of the component in the list. 

We'd like to implement a function ```groupBy(L, id)``` which takes the list of tuples ```L``` and the ```id``` of the component and returns a dictionary. The dictionary is an index very similar to what you implemented for a search engine. 
We have a key for each distinct value in column ```id```. The value of a certain key ```k``` is the list of indexes of all the tuple having value ```k``` in the column ```id```.
This means that, if index ```p``` is in the list of key ```k```, then ```L[p][id] = k```.

For example it we have tuples 

|   | 
|:-|
(1, 5, 11)
(0, 4, 1000)
(1, 2, 11)
(1, 4, 66) 
(0, 3, 99)

The groupBy with id=0 will group by first column.

The index is

|   | 
|:--| 
0: [1, 4]
1: [0, 2, 3]
 
#### Part II
We'd like to implement a function ```max_groupBy(index, L)``` which takes the index built in previous part on list ```L``` and returns a dictionary. 
We have a key for each distinct value in column ```id```. The value of a certain key ```k``` is the list. The list has a element for each column: the maximum value in that column for each tuple having value ```k``` in the column ```id```. This, of course, must be implemented by using the index.

In the example before, we would obtain the dictionary

|   | 
|:--| 
0: [0, 4, 1000]
1: [1, 5, 66]
 


In [ ]:
## Your implementation goes here

In [ ]:
## Test you implementation here

In [ ]:
data = [(1, 5, 11), (0, 4, 1000), (1, 2, 11), (1, 4, 66), (0, 3, 99)]



---

## Combining and Merging Datasets

Data contained in Pandas objects can be combined together in a number of ways:
    
- **merge()** connects rows in DataFrames based on one or more keys. This will be familiar to users of SQL or other relational databases, as it implements database **join** operations.
- **concat()** concatenates or “stacks” together objects along an axis.
- The **combine_first()** instance method enables splicing together overlapping data to fill in missing values in one object with values from another.

Here we will cover only the first two ways.

### DataFrame Joins

#### Many-to-One join
Consider the following two DataFrames.

In [10]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 
                    'data1': range(7)})

df2 = pd.DataFrame({'key': ['a', 'b', 'd'], 
                    'data2': range(3)})

print(df1)
print(df2)

  key  data1
0   b      0
1   b      1
2   a      2
3   c      3
4   a      4
5   a      5
6   b      6
  key  data2
0   a      0
1   b      1
2   d      2


The data in df1 has multiple rows labeled a and b, whereas df2 has only one row for each value in the key column. 

The operation 

```
pd.merge(df1, df2, on='key')
```

will produce a merged DataFrame with key the column to join on. 


In [11]:
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


Notice that 
- data2 of df2 is replicated everywhere key 'a' and 'b' occur in df1
- rows with key 'c' in df1 and 'd' in df2 are not present in df

The latter behavior is called **inner** join: the keys in the result are the intersection, or the common set found in both tables.

Other possible approaches are: 
- **left**: keys of left DataFrame are kept
- **right**: keys of right DataFrame are kept
- **outer**: keys of both DataFrames are kept

and can be chosen by setting parameter *how*

#### left

In [12]:
pd.merge(df1, df2, on='key', how='left')

,key,data1,data2
0,b,0,1.0
1,b,1,1.0
2,a,2,0.0
3,c,3,NaN
4,a,4,0.0
5,a,5,0.0
6,b,6,1.0


#### right

In [13]:
pd.merge(df1, df2, on='key', how='right')

,key,data1,data2
0,a,2.0,0
1,a,4.0,0
2,a,5.0,0
3,b,0.0,1
4,b,1.0,1
5,b,6.0,1
6,d,NaN,2


#### outer

In [14]:
pd.merge(df1, df2, on='key', how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


#### Many-to-Many join
Many-to-many merges happens when the same key has more than one occurrence in both DataFrames.

Many-to-many joins form the Cartesian product of the rows having the same key. 

See the following example:

In [15]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a',  'a', 'b'], 
                    'data1': range(5)})
                    
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                    'data2': range(5)})

print(df1)


print(df2)

                    
pd.merge(df1, df2, on='key', how='left')

  key  data1
0   b      0
1   b      1
2   a      2
3   a      3
4   b      4
  key  data2
0   a      0
1   b      1
2   a      2
3   b      3
4   d      4


,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,a,2,0
5,a,2,2
6,a,3,0
7,a,3,2
8,b,4,1
9,b,4,3


Note that, since there were three 'b' rows in df1 and two in df2, there are six 'b' rows in the result.

---

The merge can be performed also respect to more than one variable. To determine which key combinations will appear in the result depending on the choice of merge method, **think of the multiple keys as forming an array of tuples** to be used as a single join key.

---
### How to implement join
Join can be implemented by using three different algorithms: 
- **Nested Loops Join**: For each row in of the first dataset, check each row of the second dataset. 
- **Hash Join**: for each distinct value of the join attribute, the set of tuples in each relation which have that value. (here)[https://en.wikipedia.org/wiki/Hash_join]. This very close to the index you have built for previous exercise. 
- **Sort-merge Join**: The key idea of the sort-merge algorithm is to first sort the relations by the join attribute, so that interleaved linear scans will encounter these sets at the same time. [here](https://en.wikipedia.org/wiki/Sort-merge_join)

If the datasets are already sorted, the third approach is the fastest with margin. Otherwise, the second one is usually the fastest one. Of course, performance depends on the number of distinct value of the join attribute.

A good **optional** exercise is to implement and compare the three possible join algorithms.

In [24]:
answer = 123

f'your answer is {answer}'

'your answer is 123'

---

# References

 - **Pandas Tutorial**. https://pandas.pydata.org/pandas-docs/stable/tutorials.html
 - **Python for data analysis**. Second Edition.
   - Chapters 7, 8 and 10 
 - **Python data science handbook**. Third Edition.
   - Chapter 3
 - https://medium.com/dunder-data/selecting-subsets-of-data-in-pandas-39e811c81a0c